# Парсинг сайта «ПИШ»

Ранее я описывал как [спарсить показатели программы «Приоритет 2030»](https://psal.ru/docs/parsing/priority-2030). Парсинг показатели программа «Передовые инженерные школы» очень похож (есть, один нюанс, который явно обозначен в тексте ниже).

In [6]:
import datetime
import requests
import pandas as pd

task_id = 'PSAL-32'
data_folder = f'../data/{task_id}'
url_base = 'https://api.socio.center/engineers'

## Список университетов

Для начал сформируем список университетов:

In [2]:
url_list = f'{url_base}/schools'
schools_list_resp = requests.post(url_list)

if schools_list_resp.status_code == 200:
    shools_list_json_data = schools_list_resp.json()
    
    if shools_list_json_data['status'] == 'success':
        shools_list = pd.json_normalize(schools_list_resp.json()['data']['items'])
        
        # Сохраняем полученный список инженерных школ
        shools_list.to_csv(f"{data_folder}/shools_list.csv", index=False)
        shools_list.to_excel(f"{data_folder}/shools_list.xlsx", index=False)
    else:
        raise Exception(f"In JSON status is {shools_list_json_data['status']}")
else:
    raise Exception(f"{url_list} return status code {schools_list_resp.status_code}")

# Выводим первые 10 записей
shools_list.head(10)

,id,slug,name,shortName,logo,themes,partners,university.name,university.shortName,university.inn,university.city,university.area,university.areaCode
0,286,tpu,Интеллектуальные энергетические системы,ИнЭС,https://api.socio.center/upload/iblock/60b/c5r...,"[{'code': '16', 'name': 'Ядерная энергетика и ...","[{'code': '7024029499', 'name': 'АО «СИБИРСКИЙ...",Томский политехнический университет,ТПУ,7018007264,Томск,Сибирский,siberia
1,288,rnimu,Школа медицинской инженерии,,https://api.socio.center/upload/iblock/b90/h6v...,"[{'code': '21', 'name': 'Медицинское приборост...","[{'code': '7704859803', 'name': 'АО «ОБЪЕДИНЕН...",РНИМУ имени Н.И. Пирогова Минздрава России,ФГАОУ ВО РНИМУ им. Н.И. Пирогова Минздрава России,7728095113,Москва,Центральный,central
2,296,mipt,"Передовая инженерная школа радиолокации, радио...",,https://api.socio.center/upload/iblock/02b/mwv...,"[{'code': '28', 'name': 'Программная инженерия...","[{'code': '7731084175', 'name': 'АО «Концерн в...",Московский физико-технический институт,"МФТИ, ФИЗТЕХ",5008006211,Москва,Центральный,central
3,304,dvfu,"Институт биотехнологий, биоинженерии и пищевых...",,https://api.socio.center/upload/iblock/ea1/6se...,"[{'code': '15', 'name': 'Пищевая промышленност...","[{'code': '2520009691', 'name': 'ООО «Арника-Х...",Дальневосточный федеральный университет,ДВФУ,2536014538,Владивосток,Дальневосточный,vostok
4,310,bmstu,Системная инженерия ракетно- космической техники,,https://api.socio.center/upload/iblock/880/yt4...,"[{'code': '17', 'name': 'Авиационная и ракетно...","[{'code': '7702388027', 'name': 'Госкорпорация...",Московский государственный технический универс...,МГТУ им. Н.Э. Баумана,7701002520,Москва,Центральный,central
5,311,sfedu,Инженерия киберплатформ,,https://api.socio.center/upload/iblock/1e0/ai5...,"[{'code': '24', 'name': 'Искусственный интелле...","[{'code': '7704274402', 'name': 'Государственн...",Южный федеральный университет,Южный федеральный университет,6163027810,Ростов-На-Дону,Южный,south
6,318,nntu,Передовая инженерная школа атомного машиностро...,,https://api.socio.center/upload/iblock/2d5/856...,"[{'code': '18', 'name': 'Атомное машиностроени...","[{'code': '5260174449', 'name': 'Региональное ...",Нижегородский государственный технический унив...,НГТУ,5260001439,Нижний Новгород,Приволжский,volga
7,319,ssau,Интегрированные технологии в создании аэрокосм...,,https://api.socio.center/upload/iblock/2c5/lxs...,"[{'code': '17', 'name': 'Авиационная и ракетно...","[{'code': '6312139922', 'name': 'АО «Ракетно-к...",Самарский университет,Самарский университет,6316000632,Самара,Приволжский,volga
8,328,tsu,Агробиотек,,https://api.socio.center/upload/iblock/7f8/8cc...,"[{'code': '9', 'name': 'Биология и биотехнолог...","[{'code': '7017012254', 'name': 'АО «СИБАГРО»'...",Томский государственный университет,ТГУ,7018012970,Томск,Сибирский,siberia
9,330,urfu,"Уральская передовая инженерная школа ""Цифровое...",УПИШ,https://api.socio.center/upload/iblock/2b3/h70...,"[{'code': '28', 'name': 'Программная инженерия...","[{'code': '6673100680', 'name': 'АО «Уральский...",Уральский федеральный университет,УрФУ,6660003190,Екатеринбург,Уральский,ural


## Показатели программы

Обратите **особое внимание** на комментарий про фикс баги. Дело в том, что API, а далее и графики на сайте ПИШ-а выводят план и факт на одной кривой, не отделяя одно от другого 🤷‍♂️

In [7]:
kpi_df = pd.DataFrame()

for i, r in shools_list.iterrows():
    slug = r['slug'].lower()
    url = f'{url_base}/school/{slug}/indicators'
    kpi_resp = requests.get(url)
    kpi_json = kpi_resp.json()

    for group in kpi_json['data']:
        group_title = group['title']

        for item in group['items']:
            kpi_chunk = pd.DataFrame(columns=['plan', 'fact'])
            
            # Данный цикл — фикс баги API сайта Социоцентра
            for idx, year in enumerate(item['label']):
                if year not in kpi_chunk.index:
                    kpi_chunk.loc[year, 'plan'] = item['value'][idx]
                else:
                    kpi_chunk.loc[year, 'fact'] = item['value'][idx]

            kpi_chunk = kpi_chunk.reset_index().rename(columns={'index': 'year'})
            kpi_chunk[['slug', 'group', 'indicator', 'unit', 'description', 'calculation_method']] = [
                slug,
                group_title,
                item['indicator'],
                item['unit'],
                item['description'],
                item['calculationMethod']
            ]

            kpi_df = pd.concat([kpi_df, kpi_chunk], ignore_index=True)

schools_kpis = shools_list.merge(kpi_df, how='right', on='slug')

# Сохраняем полученные KPI
schools_kpis.to_csv(f"{data_folder}/schools_kpis.csv", index=False)
schools_kpis.to_excel(f"{data_folder}/schools_kpis.xlsx", index=False)

Выводим пример основных данных связанных с KPI университетов за прошедшие года:

In [13]:
schools_kpis[schools_kpis['year'] <= datetime.datetime.now().year][[
    'university.shortName',
    'indicator',
    'year',
    'plan',
    'fact',
    'unit'
]].sort_values(['indicator', 'university.shortName', 'year']).head(10)

,university.shortName,indicator,year,plan,fact,unit
5210,АНО ВО «Университет Иннополис»,М__т_6_1__с_03__г_3__ц_47_ПИШ,2021,0,NaN,тыс. руб.
5211,АНО ВО «Университет Иннополис»,М__т_6_1__с_03__г_3__ц_47_ПИШ,2022,250000,NaN,тыс. руб.
5212,АНО ВО «Университет Иннополис»,М__т_6_1__с_03__г_3__ц_47_ПИШ,2023,250000,348221.871,тыс. руб.
5213,АНО ВО «Университет Иннополис»,М__т_6_1__с_03__г_3__ц_47_ПИШ,2024,250000,NaN,тыс. руб.
5970,ГБОУ ВО АГТУ ВШН,М__т_6_1__с_03__г_3__ц_47_ПИШ,2021,0,NaN,тыс. руб.
5971,ГБОУ ВО АГТУ ВШН,М__т_6_1__с_03__г_3__ц_47_ПИШ,2022,0,NaN,тыс. руб.
5972,ГБОУ ВО АГТУ ВШН,М__т_6_1__с_03__г_3__ц_47_ПИШ,2023,0,NaN,тыс. руб.
5973,ГБОУ ВО АГТУ ВШН,М__т_6_1__с_03__г_3__ц_47_ПИШ,2024,149370,NaN,тыс. руб.
6350,ГГНТУ им. акад. М.Д. Миллионщикова,М__т_6_1__с_03__г_3__ц_47_ПИШ,2021,0,NaN,тыс. руб.
6351,ГГНТУ им. акад. М.Д. Миллионщикова,М__т_6_1__с_03__г_3__ц_47_ПИШ,2022,0,NaN,тыс. руб.
